In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.graphics.api import qqplot
import math
import time
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error


import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
rcParams['figure.figsize'] = 30, 10
print('setup complete')


setup complete


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
conf_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
reco_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [3]:
import datetime

In [4]:
#getting all the dates
cols = conf_df.keys() #it has all the columns

conf = conf_df.loc[:, cols[4]:cols[-1]]
death = death_df.loc[:, cols[4]:cols[-1]]
reco = reco_df.loc[:, cols[4]:cols[-1]]

In [5]:
#now we have all the dates
dates = conf.keys()

In [6]:
dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20',
       '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20',
       '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20',
       '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20'],
      dtype='object')

In [7]:
world_cases = []
total_deaths = [] 
mortality_rate = []
recovery_rate = [] 
total_recovered = [] 
total_active = [] 

countrywise_cases_conf = []
countrywise_cases_reco = []
countrywise_cases_death = []
countrywise_cases_active = []

#j=0
for i in dates:
    
    confirmed_sum = conf[i].sum()
    death_sum = death[i].sum()
    recovered_sum = reco[i].sum()
    
    # confirmed, deaths, recovered, and active
    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)
    total_recovered.append(recovered_sum)
    total_active.append(confirmed_sum-death_sum-recovered_sum)
    
    # calculate rates
    
    # case studies 
    #china_cases.append(conf_df[conf_df['Country/Region']=='China'][i].sum())
    #italy_cases.append(conf_df[conf_df['Country/Region']=='Italy'][i].sum())
#     india_cases_conf.append(conf_df[conf_df['Country/Region']=='India'][i].sum())
#     india_cases_reco.append(reco_df[reco_df['Country/Region']=='India'][i].sum())
#     india_cases_death.append(death_df[death_df['Country/Region']=='India'][i].sum())
#     india_cases_active.append(india_cases_conf[j] - india_cases_reco[j] - india_cases_death[j])
    
#    j = j+1

In [8]:
days_in_future = 10
future_forcast = np.array([i for i in range(len(dates)+days_in_future)]).reshape(-1, 1)
adjusted_dates = future_forcast[:-10]

In [9]:
days_since_1_22 = np.array([i for i in range(len(dates))]).reshape(-1, 1)
world_cases = np.array(world_cases).reshape(-1, 1)
total_deaths = np.array(total_deaths).reshape(-1, 1)
total_recovered = np.array(total_recovered).reshape(-1, 1)

In [10]:
start = '1/22/2020'
start_date = datetime.datetime.strptime(start, '%m/%d/%Y')
future_forcast_dates = []
for i in range(len(future_forcast)):
    future_forcast_dates.append((start_date + datetime.timedelta(days=i)).strftime('%m/%d/%Y'))

In [ ]:
#total confirmed cases in the world
world_df = pd.DataFrame(index=dates,data=world_cases, columns=['conf_cases'])
world_df['daily_new_cases'] = world_df.diff()
world_df['daily_new_cases'][0] = world_df['conf_cases'][0]
world_df['daily_new_cases'] =world_df['daily_new_cases'].astype('int64')

In [ ]:
total_death_df = pd.DataFrame(index=dates,data=total_deaths, columns=['total_death'])
total_death_df['new_death'] = total_death_df.diff()
total_death_df['new_death'][0] = total_death_df['total_death'][0] 
total_death_df['new_death'] = total_death_df['new_death'].astype('int64')

In [ ]:
total_reco_df = pd.DataFrame(index=dates,data=total_recovered, columns=['total_recovered'])
total_reco_df['new_reco'] = total_reco_df.diff()
total_reco_df['new_reco'][0] = total_reco_df['total_recovered'][0] 
total_reco_df['new_reco'] = total_reco_df['new_reco'].astype('int64')

In [ ]:

##this is tricky but true as it is daily new active cases being found both way it give same ans
total_active_df = pd.DataFrame(index=dates,data=total_active, columns=['total_active'])

#total_active_df['new_active111'] = world_df['daily_new_cases'] - total_death_df['new_death'] - total_reco_df['new_reco']
total_active_df['new_active'] = total_active_df['total_active'].diff()
total_active_df['new_active'][0] = total_active_df['total_active'][0] 
total_active_df['new_active'] = total_active_df['new_active'].astype('int64')

total_active_df=total_active_df.clip(lower =0)

In [15]:
world_full = pd.concat([world_df,total_reco_df,total_death_df,total_active_df], axis=1)

In [16]:
#this dataset is for regression for univariate 

In [17]:
world_full.head(5) #datasetprepared

,conf_cases,daily_new_cases,total_recovered,new_reco,total_death,new_death,total_active,new_active
1/22/20,555.0,555,28.0,28,17.0,17,510.0,510
1/23/20,653.0,98,30.0,2,18.0,1,605.0,95
1/24/20,941.0,288,36.0,6,26.0,8,879.0,274
1/25/20,1434.0,493,39.0,3,42.0,16,1353.0,474
1/26/20,2118.0,684,52.0,13,56.0,14,2010.0,657


In [18]:
world_full.index

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20',
       '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20',
       '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20',
       '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20'],
      dtype='object')

# Prediction 
for next 10 days

### Solving It as Univariate Problem

In [19]:
X_train_confirmed, X_test_confirmed, y_train_confirmed, y_test_confirmed = train_test_split(days_since_1_22, world_cases, test_size=0.15, shuffle=False) 

In [ ]:
test_stationarity(tsdata)

 Mean and standard deviation are clearly increasing with time and this is not a stationary series. Also, the test statistic is way more than the critical values. Note that the signed values should be compared and not the absolute values.

## Making Time Series stationary

1. Trend 
2. Seasonality 

### Estimating & Eliminating Trend

In [ ]:
ts_log = np.log(ts)
plt.plot(ts_log)

In [ ]:
ts_log.index.dtype

### Moving Averages


In [ ]:
#ts_log = pd.DataFrame(ts_log)
moving_avg = ts_log.rolling(7).mean()
plt.plot(ts_log)
plt.plot(moving_avg, color='red')

In [ ]:
ts_log_moving_avg_diff = ts_log - moving_avg

In [ ]:
ts_log_moving_avg_diff.head(10) #as for first 7 values there is no rolling value
#hence dropping it
ts_log_moving_avg_diff.dropna(inplace=True)
#checking stationarity
test_stationarity(ts_log_moving_avg_diff)

This looks like a much better series. The rolling values appear to be varying but there is no specific trend. Also, the test statistic is smaller than the 1% critical values so we can say with 99% confidence that this is a stationary series.

## Weighted Moving Average
Recent data gets more weightage

In [ ]:
expwighted_avg = ts_log.ewm(span=7).mean() #span is deacy factor

plt.plot(ts_log)
plt.plot(expwighted_avg, color='red')

In [ ]:
ts_log_ewma_diff = ts_log - expwighted_avg
test_stationarity(ts_log_ewma_diff)

Test Statistic  is much lagrger hence previous case was better

### Differencing

In [ ]:
ts_log_diff = ts_log - ts_log.shift()
plt.plot(ts_log_diff)

In [ ]:
ts_log

In [ ]:
ts_log_diff.dropna(inplace=True)
test_stationarity(ts_log_diff)

No success with differencing also

In [ ]:
ts_log_diff2 = ts_log_diff - ts_log_diff.shift()
plt.plot(ts_log_diff2)

In [ ]:
ts_log_diff2.dropna(inplace=True)
test_stationarity(ts_log_diff2)

Second order differencing gives promising results

### Forecasting a Time Series

In [ ]:
#ACF and PACF plots:
from statsmodels.tsa.stattools import acf, pacf
lag_acf = acf(ts_log_diff2, nlags=5)
lag_pacf = pacf(ts_log_diff2, nlags=5, method='ols')
#Plot ACF: 
plt.subplot(121) 
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_log_diff2)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_log_diff2)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')
#Plot PACF:
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_log_diff2)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_log_diff2)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

1. p – The lag value where the PACF chart crosses the upper confidence interval for the first time. If you notice closely, in this case p=1.
2. q – The lag value where the ACF chart crosses the upper confidence interval for the first time. If you notice closely, in this case q=1

In [ ]:
from __future__ import print_function
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm

from statsmodels.graphics.api import qqplot


In [ ]:
# try plotting for various lags 2,3,4,5...20
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)

fig = sm.graphics.tsa.plot_acf(ts_log_diff2.values.squeeze(), lags=3, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(ts_log_diff2, lags=3, ax=ax2)

In [ ]:
train = ts_log_diff2.iloc[0:50]

In [ ]:
test = ts_log_diff2.iloc[50:]

In [ ]:
print (len(train))
print (len(test))

#### We need to load the ARIMA model first:

In [ ]:
from statsmodels.tsa.arima_model import ARIMA


The p,d,q values can be specified using the order argument of ARIMA which take a tuple (p,d,q).
 p is the number of autoregressive terms, d is the number of nonseasonal differences needed for stationarity, and. q is the number of lagged forecast errors in the prediction equation.

### AR model:

In [ ]:
model = ARIMA(ts_log, order=(2, 2, 0)) #d=2 because 2 oder difference gave best stationarity test  
results_AR = model.fit(disp=-1)  
plt.plot(ts_log_diff2)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_AR.fittedvalues-ts_log_diff2)**2))

### Moving Average Model

In [ ]:
model = ARIMA(ts_log, order=(0, 2, 2))  
results_MA = model.fit(disp=-1)  
plt.plot(ts_log_diff2)
plt.plot(results_MA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_MA.fittedvalues-ts_log_diff2)**2))

### Combined model:

In [ ]:
model = ARIMA(ts_log, order=(2, 2, 2))  
results_ARIMA = model.fit(disp=-1)  
plt.plot(ts_log_diff2)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues-ts_log_diff2)**2))

### Combined AR and MA work better 

### Using Pyramid ARIMA 
It has auto arima 

In [ ]:
import pyramid as pm
from pyramid.arima import auto_arima
world_df.head()

In [ ]:
#load the data
data = world_df['conf_cases'].copy()

#divide into train and validation set
#train = data[:int(0.7*(len(data)))]
#valid = data[int(0.7*(len(data))):]

#preprocessing (since arima takes univariate series as input)
#train.drop('Month',axis=1,inplace=True)
#valid.drop('Month',axis=1,inplace=True)

#plotting the data
train.plot()
valid.plot()

In [ ]:
from pyramid.arima import auto_arima
model = auto_arima(data, seasonal=[True,False],seasonal_decompose=[True,False], trace=True, error_action='ignore', suppress_warnings=True)
model.fit(train)

forecast = model.predict(n_periods=len(future_forcast_dates))
forecast = pd.DataFrame(forecast,index = future_forcast_dates.index,columns=['Prediction'])

#plot the predictions for validation set
plt.plot(train, label='Train')
#plt.plot(valid, label='Valid')
plt.plot(forecast, label='Prediction')
plt.show()

Hence Its not much useful to use ARIMA on this dataset

In [ ]:
# plt.plot(y_test_confirmed)
# plt.plot(test_bayesian_pred)